In [4]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score

from sklearn.linear_model import HuberRegressor
pd.set_option('display.max_rows', None)
# more

In [6]:
# df = pd.read_csv('factors_2002.csv', parse_dates=['DATE'])
# df = pd.read_csv('data/features_subset.csv', parse_dates=['DATE'])
df = pd.read_csv('factors_1900.csv', parse_dates=['DATE'])


In [7]:
#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,macro_dp,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar,Date,macro_mkt-rf
0,10001,1990-01-31,10347.750000,0.021663,0.000469,0.348191,8.308599,0.037859,0.179456,0.037975,...,3.385516,2.68412,0.414971,-0.013897,0.0764,-0.1072,0.0095,0.002892,199004.0,-7.85
1,10002,1990-01-31,7196.875000,0.481378,0.231724,-0.268657,6.680698,0.018793,0.127571,-0.039216,...,3.385516,2.68412,0.414971,-0.013897,0.0764,-0.1072,0.0095,0.002892,199004.0,-7.85
2,10003,1990-01-31,18649.000000,0.765364,0.585782,-0.281724,9.927838,0.040525,0.127571,-0.260870,...,3.385516,2.68412,0.414971,-0.013897,0.0764,-0.1072,0.0095,0.002892,199004.0,-7.85
3,10005,1990-01-31,2043.505859,0.980050,0.960498,-0.033333,5.100628,0.148197,0.336596,0.000000,...,3.385516,2.68412,0.414971,-0.013897,0.0764,-0.1072,0.0095,0.002892,199004.0,-7.85
4,10007,1990-01-31,28196.250000,1.672263,2.796464,1.210168,11.458924,0.188345,0.174430,0.450331,...,3.385516,2.68412,0.414971,-0.013897,0.0764,-0.1072,0.0095,0.002892,199004.0,-7.85


In [16]:

df= df[~np.isnan(df['bm'])]
df =df[~np.isnan(df['mvel1'])]

In [8]:

df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [15]:
p=0.3 
df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


C:\Users\drebi\AppData\Local\Temp\ipykernel_39752\2840026024.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)
C:\Users\drebi\AppData\Local\Temp\ipykernel_39752\2840026024.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1'))

In [10]:
#Standardize all independent variables

stdSc = StandardScaler()

features = df.columns[~df.columns.isin(['DATE', 'DATE2', 'mvel1', 'mvel12', 'permno', 'permno2', 'risk_premium'])].tolist()
df[features] = stdSc.fit_transform(df[features].astype(float))


In [11]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


X = df[['DATE', 'macro_mkt-rf', 'macro_tbl', 'mvel1', 'bm']]
y = df[['risk_premium']]

#Empty containers to save results from each window

predictions = []
y_train_list = []
y_val_list = []
y_test_list =[]
dates = []
dic_r2_all = {}

lm = LinearRegression()

for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(1995,1,31), second_split_date= datetime.date(1997,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))


    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]
    y_pred_val = lm.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))


    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    y_pred_test = lm.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))
    
   
    
    
# predictions_all= np.concatenate(predictions, axis=0)
# y_test_list_all= np.concatenate(y_test_list, axis=0) 
# dates_all= np.concatenate(dates, axis=0)

# R2OOS_LR = 1-sum(pow(y_test_list_all-predictions_all,2))/sum(pow(y_test_list_all,2))
# print("R2OOS Huber Regression: ", R2OOS_LR)


Train period: 1990-01-31 - 1995-01-31 ,val period: 1995-01-31 - 1997-01-31 , Test period 1997-01-31 - 1998-01-31 # train records 424371 ,# val records 202339 , # test records 117085
Train period: 1991-01-31 - 1996-01-31 ,val period: 1996-01-31 - 1998-01-31 , Test period 1998-01-31 - 1999-01-31 # train records 441982 ,# val records 221030 , # test records 106363
Train period: 1992-01-31 - 1997-01-31 ,val period: 1997-01-31 - 1999-01-31 , Test period 1999-01-31 - 2000-01-31 # train records 466137 ,# val records 223448 , # test records 92348
Train period: 1993-01-31 - 1998-01-31 ,val period: 1998-01-31 - 2000-01-31 , Test period 2000-01-31 - 2001-01-31 # train records 494688 ,# val records 198711 , # test records 98810
Train period: 1994-01-31 - 1999-01-31 ,val period: 1999-01-31 - 2001-01-31 , Test period 2001-01-31 - 2002-01-31 # train records 521423 ,# val records 191158 , # test records 92269
Train period: 1995-01-31 - 2000-01-31 ,val period: 2000-01-31 - 2002-01-31 , Test period 2002

In [12]:
y_val_list

[0.02851388869981164,
 0.03897198427655235,
 0.07668141337841594,
 0.06639710866742043,
 0.043995405466819903,
 0.08372081716793045,
 0.09571086166087939,
 0.10824770316262411,
 0.06444195540920461,
 0.05848049114320364,
 0.05589692618388287,
 0.009295466972519262,
 0.09027796978096048,
 0.1921132437836005,
 0.16157648015808534,
 0.1819455383769929,
 0.1362504982670245,
 0.07092569824664108,
 0.049540233797152444,
 0.06183973424653155,
 0.07551049943868704,
 0.03114938391260813,
 0.03407622585163328,
 0.11838974997038243]

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


X = df_large[['DATE', 'macro_mkt-rf', 'macro_tbl', 'mvel1', 'bm']]
y = df_large[['risk_premium']]

#Empty containers to save results from each window

predictions = []
y_train_list = []
y_val_list = []
y_test_list =[]
dates = []
dic_r2_all = {}

lm = LinearRegression()

for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(1995,1,31), second_split_date= datetime.date(1997,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))


    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]
    y_pred_val = lm.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))


    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    y_pred_test = lm.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))
    
   
    
    
# predictions_all= np.concatenate(predictions, axis=0)
# y_test_list_all= np.concatenate(y_test_list, axis=0) 
# dates_all= np.concatenate(dates, axis=0)

# R2OOS_LR = 1-sum(pow(y_test_list_all-predictions_all,2))/sum(pow(y_test_list_all,2))
# print("R2OOS Huber Regression: ", R2OOS_LR)


Train period: 1990-01-31 - 1995-01-31 ,val period: 1995-01-31 - 1997-01-31 , Test period 1997-01-31 - 1998-01-31 # train records 127287 ,# val records 60692 , # test records 35119
Train period: 1991-01-31 - 1996-01-31 ,val period: 1996-01-31 - 1998-01-31 , Test period 1998-01-31 - 1999-01-31 # train records 132570 ,# val records 66297 , # test records 31901
Train period: 1992-01-31 - 1997-01-31 ,val period: 1997-01-31 - 1999-01-31 , Test period 1999-01-31 - 2000-01-31 # train records 139816 ,# val records 67020 , # test records 27698
Train period: 1993-01-31 - 1998-01-31 ,val period: 1998-01-31 - 2000-01-31 , Test period 2000-01-31 - 2001-01-31 # train records 148379 ,# val records 59599 , # test records 29639
Train period: 1994-01-31 - 1999-01-31 ,val period: 1999-01-31 - 2001-01-31 , Test period 2001-01-31 - 2002-01-31 # train records 156397 ,# val records 57337 , # test records 27676
Train period: 1995-01-31 - 2000-01-31 ,val period: 2000-01-31 - 2002-01-31 , Test period 2002-01-31 

In [16]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


X = df_small[['DATE', 'macro_mkt-rf', 'macro_tbl', 'mvel1', 'bm']]
y = df_small[['risk_premium']]

#Empty containers to save results from each window

predictions = []
y_train_list = []
y_val_list = []
y_test_list =[]
dates = []
dic_r2_all = {}

lm = LinearRegression()

for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(1995,1,31), second_split_date= datetime.date(1997,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))


    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]
    y_pred_val = lm.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))


    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    y_pred_test = lm.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))
    
   
    
    
# predictions_all= np.concatenate(predictions, axis=0)
# y_test_list_all= np.concatenate(y_test_list, axis=0) 
# dates_all= np.concatenate(dates, axis=0)

# R2OOS_LR = 1-sum(pow(y_test_list_all-predictions_all,2))/sum(pow(y_test_list_all,2))
# print("R2OOS Huber Regression: ", R2OOS_LR)


Train period: 1990-01-31 - 1995-01-31 ,val period: 1995-01-31 - 1997-01-31 , Test period 1997-01-31 - 1998-01-31 # train records 127287 ,# val records 60692 , # test records 35119
Train period: 1991-01-31 - 1996-01-31 ,val period: 1996-01-31 - 1998-01-31 , Test period 1998-01-31 - 1999-01-31 # train records 132570 ,# val records 66297 , # test records 31901
Train period: 1992-01-31 - 1997-01-31 ,val period: 1997-01-31 - 1999-01-31 , Test period 1999-01-31 - 2000-01-31 # train records 139816 ,# val records 67020 , # test records 27698
Train period: 1993-01-31 - 1998-01-31 ,val period: 1998-01-31 - 2000-01-31 , Test period 2000-01-31 - 2001-01-31 # train records 148379 ,# val records 59599 , # test records 29639
Train period: 1994-01-31 - 1999-01-31 ,val period: 1999-01-31 - 2001-01-31 , Test period 2001-01-31 - 2002-01-31 # train records 156397 ,# val records 57337 , # test records 27676
Train period: 1995-01-31 - 2000-01-31 ,val period: 2000-01-31 - 2002-01-31 , Test period 2002-01-31 

In [17]:
y_val_list

[0.005734610789250083,
 0.015300596373754916,
 0.032773325597783454,
 0.030929975427823275,
 0.02584453168765999,
 0.061932216127773665,
 0.04467546584204518,
 0.07335300997397365,
 0.032431027445364746,
 0.02680421077790729,
 0.02742491357052912,
 -0.02147780690255341,
 0.019380193296187453,
 0.11672741918184737,
 0.08343769392665568,
 0.08525002622243538,
 0.06681913434125386,
 0.04318338214076556,
 0.013910283311432625,
 0.00871773837534795,
 0.02089629957259853,
 0.007490697533350521,
 0.05388564930531747,
 0.07048617187392159]